#Kaggle에서 실습할 경우
① Add data 클릭 → ② "metal surface defects" 검색 → ③ Add 클릭

이후 우측 상단 input 폴더를 클릭하여  "neu-metal-surface-defects-data" 가 추가된 것을 확인

#사내 PC에서 실습할 경우
아래 링크로 데이터를 다운로드 받은 후, D드라이브 작업방에 압축을 해제해 놓으세요.

데이터셋 : https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data/download

### 데이터 로드

In [ ]:
#모듈 불러오기
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#root dir 설정
# root_dir = "../input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data" #캐글에서 실습할 경우
root_dir = "D:/작업방/NEU Metal Surface Defects Data" #PC에서 실습할 경우

#os.listdir() 코드로 폴더 탐색
os.listdir(root_dir + "/train")

In [ ]:
#os.listdir() 코드로 폴더 탐색
os.listdir(root_dir + "/train/Inclusion")

In [ ]:
#폴더명 지정하여 변수로 저장 
train_dir = root_dir + "/train"
val_dir = root_dir + "/valid"
test_dir = root_dir + "/test"

#라벨명을 리스트로 미리 만들어 놓기
labels = os.listdir(train_dir)
labels

In [ ]:
#train 하위 폴더별 이미지 개수 파악
for label in labels:
    print("{} : {}개".format(label, len(os.listdir(train_dir+"/"+label))))

### 이미지 확인

In [ ]:
#이미지 하나 불러와서 출력하기
img_file_list = os.listdir(train_dir+"/"+labels[0]+"/")
img_file = train_dir+"/"+labels[0]+"/"+img_file_list[0]
image_pil = Image.open(img_file)
image_pil

In [ ]:
#이미지를 numpy ndarray로 변환
image = np.array(image_pil)
print(image.shape)  #이미지 모양 200 X 200
image

In [ ]:
#numpy ndarray로 변환된 이미지로 pyplot으로 출력
plt.imshow(image, "gray")
plt.show()

### Image Augmentation

In [ ]:
#tensorflow 이미지데이터 제너레이터 객체 생성(Train 데이터셋)
train_datagen = ImageDataGenerator(
    rescale=1./255, #0~1사이 값으로 리스케일링
    shear_range=0.2, #시계반대방향으로 밀림 강도를 0 ~ 0.2 사이로 조정
    zoom_range=0.2, #원본 이미지 확대 축소 (1-값 ~ 1+값), 0.8 ~ 1.2 로 조정
    horizontal_flip=True #좌/우 뒤집기. vertical_flip = True 추가하면 상/하 뒤집기
)

#flow_from_directory 메소드로 증강된 데이터셋을 생성해 주는 객체 생성
train_generator = train_datagen.flow_from_directory( #Takes the path to a directory & generates batches of augmented data.
    train_dir,
    target_size=(200, 200), #생성할 이미지 사이즈를 200 X 200으로 설정
    batch_size=10, #생성할 이미지의 배치사이즈(묶음)를 10개로 설정
    class_mode="categorical" #생성할 이미지의 용도를 다분류 모델로 지정 (생략가능)
) #출력결과 : Found 1656 images belonging to 6 classes.

len(train_generator) #출력결과 : 166묶음 ← 1656/10(이미지 총개수 1656 / 배치사이즈 10)

In [ ]:
#생성되는 이미지 확인하기
#train_generator 객체를 반복문으로 접근하면 변형된 이미지를 생성하고 배치사이즈만큼 묶어 출력해 줌(무한번 반복)
for datas in train_generator:
    imgs = datas[0] #datas의 0번째 리턴값은 생성한 이미지 묶음(10, 200, 200, 3)
    label_per_imgs = datas[1] #datas의 1번째 리턴값은 생성한 이미지들의 라벨값(10, 6). one-hot encoding으로 출력
    print(len(datas), imgs.shape, label_per_imgs.shape)
    for img in imgs:
        plt.imshow(img, "gray")
        plt.show()
    break

In [ ]:
#Validation 데이터셋 또한 이미지데이터 제너레이터 객체를 생성해 데이터 증강
val_datagen = ImageDataGenerator(rescale=1./255)

#검증용데이터 디렉토리, 배치, 이미지크기 지정
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(200, 200),
        batch_size=10,
        class_mode="categorical"
)

len(validation_generator)

### 모델 생성

In [ ]:
#모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(200, 200, 3), kernel_size=(3, 3), filters=32, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=6, activation="softmax")
])

model.summary() #모델 확인

#모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"]
)

In [ ]:
#모델 학습
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

In [ ]:
#트레이닝 결과 확인
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()

plt.show()

In [ ]:
#케라스 모델로 저장
save_path = "my_model.h5"
model.save(save_path, include_optimizer=True)

### 모델 활용

Kaggle에서 모델을 만든 경우, 아래 절차대로 모델을 다운로드 받은 후 코드를 실행해 주세요.


1. 우측 메뉴에 있는 "/kaggle/working" 폴더를 클릭

2. my_model.h5 우측으로 마우스 이동 후 점세개 모양 버튼 클릭 후 다운로드 클릭

3. pc의 "d:/작업방" 폴더에 "ai" 폴더 생성 (다른 폴더도 상관없음)

4. 캐글에서 다운로드 받은 "my_model.h5" 파일을 "ai" 폴더에 복사

5. pc의 "ai" 폴더 하위에 test_images 폴더 생성 

6. github test_images 폴더에 있는 이미지 파일 16개를 "ai/test_images" 폴더에 복사

7. "ai" 폴더에서 jupyter notebook 실행 

In [ ]:
#모델 불러오기
model = tf.keras.models.load_model("my_model.h5")
model.summary()

In [ ]:
#test 이미지 1개 확인
img_file_list = os.listdir(test_dir+"/"+labels[0]+"/")
img_file = test_dir+"/"+labels[0]+"/"+img_file_list[0]
image_pil = Image.open(img_file).convert("RGB") #컬러 이미지로 불러오기

img = np.array(image_pil) #array로 변환
img = img/255. #0~1 사이로 리스케일링
img = img[tf.newaxis, ...] #앞쪽에 배열 추가,  1,200,200,3 
pred = model.predict(img) #모델을 활용하여 img 예측, pred는 6개의 확률값
classes = np.argmax(pred) #첫번째 배열에서 가장 큰 값의 위치 찾기
confidence = round(np.max(pred)*100,2) #확률값 계산
names = labels[classes] #class_names 찾기
title = "{} : {}".format(names, confidence) #타이틀 텍스트 만들기	

plt.title(title)
plt.imshow(np.squeeze(img), "gray") #200,200 이미지 출력
plt.show()

In [ ]:
#테스트할 이미지 파일을 뽑아 리스트 변수에 추가
test_file_list = []
for dir in os.listdir(test_dir): #test 폴더 하위에 있는 6개 폴더를 반복문으로 접근
    for idx, file in enumerate(os.listdir(test_dir+"/"+dir)): #6개 폴더별 파일들에 반복문으로 접근
        if idx < 4: #클래스별 4개 파일만 뽑아내 test_file_list 변수에 추가
            test_file_list.append(test_dir+"/"+dir+"/"+file)
test_file_list #클래스별 4개 * 6개 클래스 = 총 24개의 이미지 경로가 저장됨

In [ ]:
#test 이미지 24건을 분류, 확률값이 80% 이하일 경우 적색 텍스트로 출력 
i=1
plt.figure(figsize=(16, 16))
for file in test_file_list:
    image_pil = Image.open(file).convert("RGB")
    img = np.array(image_pil)
    img = img/255.
    img = img[tf.newaxis, ...]
    pred = model.predict(img)
    classes = np.argmax(pred)
    confidence = round(np.max(pred)*100, 2)
    names = labels[classes]
    title = "{} : {}% (True : {})".format(names, confidence, file.split("/")[-1].split("_")[0])
    plt.subplot(6, 4, i)
    if confidence < 80:
        plt.title(title, color="red")
    else:
        plt.title(title, color="blue")
    plt.axis("off")
    plt.imshow(np.squeeze(img), "gray")
    i += 1
plt.show()